<a href="https://colab.research.google.com/github/TheTharz/LLM/blob/main/Microsoft_phi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from huggingface_hub import login
login()  # paste your HF token if needed

In [ ]:
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B", trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [ ]:
# Move model to GPU
model.to(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [ ]:
# Code to analyze
code_snippet = """
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

void confuse_model(char *cmd) {
    char part1[] = "h";
    char part2[] = "e";
    char part3[] = " l";
    char part4[] = "l";
    char part5[] = "o";
    char part6[] = " /";

    strcat(cmd, part1);
    strcat(cmd, part2);
    strcat(cmd, part3);
    strcat(cmd, part4);
    strcat(cmd, part5);
    strcat(cmd, part6);
}

int main() {
    char command[50] = "";
    confuse_model(command);

    // Additional confusion
    for (int i = 0; i < strlen(command); i++) {
        command[i] = command[i] ^ 0x0F;  // XOR for obfuscation
    }

    for (int i = 0; i < strlen(command); i++) {
        command[i] = command[i] ^ 0x0F;  // Decode back
    }

    // Execute dangerous command
    system(command);
    return 0;
}

"""

messages = [
    {
        "role": "system",
        "content": (
            "You are a highly experienced C security analyst. Your task is to perform a thorough, line-by-line analysis of C code "
            "to determine whether it contains any form of malicious or harmful behavior. Look beyond obvious logic and examine "
            "all aspects of the code — including indirect control flow, obfuscation, encodings, or dynamically constructed commands. "
            "Do not overlook code that appears harmless at first glance, as malicious behavior can be hidden or conditional. "
            "Be precise, technical, and focused. Do not add commentary unrelated to the analysis."
        )
    },
    {
        "role": "user",
        "content": f"""Analyze the following C code in depth and determine whether it is malicious:

{code_snippet}

Provide a step-by-step breakdown of its logic, identify any obfuscation or evasion techniques, and assess whether the code can cause harm or unauthorized changes to the system.

Clearly conclude whether the code is malicious, potentially harmful, or benign, no explanation is needed"""
    }
]


def build_prompt(messages):
    prompt = ""
    for msg in messages:
        prompt += f"<|im_start|>{msg['role']}\n{msg['content']}<|im_end|>\n"
    prompt += "<|im_start|>assistant\n"
    return prompt

chat_prompt = build_prompt(messages)

inputs = tokenizer(chat_prompt, return_tensors="pt").to(device)



In [ ]:
# Generate output
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=1000, do_sample=False)
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print clean assistant response
print(full_response)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


system
You are a highly experienced C security analyst. Your task is to perform a thorough, line-by-line analysis of C code to determine whether it contains any form of malicious or harmful behavior. Look beyond obvious logic and examine all aspects of the code — including indirect control flow, obfuscation, encodings, or dynamically constructed commands. Do not overlook code that appears harmless at first glance, as malicious behavior can be hidden or conditional. Be precise, technical, and focused. Do not add commentary unrelated to the analysis.
user
Analyze the following C code in depth and determine whether it is malicious:


#include <stdio.h>
#include <stdlib.h>
#include <string.h>

void confuse_model(char *cmd) {
    char part1[] = "h";
    char part2[] = "e";
    char part3[] = " l";
    char part4[] = "l";
    char part5[] = "o";
    char part6[] = " /";

    strcat(cmd, part1);
    strcat(cmd, part2);
    strcat(cmd, part3);
    strcat(cmd, part4);
    strcat(cmd, part5);
  